In [3]:
import brickschema
from brickschema.namespaces import BRICK, RDFS, OWL, TAG, RDF, SH
from rdflib import URIRef
import networkx

from utils import *
from rules.relationship_relaxation import *
from rules.type_relaxation import *
from relaxation_graphs.fast_relaxation_graph import *
from evaluation.metrics_fast import *
from evaluation.similarity_score import *

# Not ideal visual just validation
import matplotlib.pyplot as plt
import networkx as nx
import datetime
import os
import pickle
import json

import pymortar
from queries.mortar import queries as q1
from queries.cbe import queries as q2
from queries.nrel import queries as q3

brick_graph = brickschema.Graph(load_brick=True)
query_bank = q1 + q2 + q3

In [ ]:
files = os.listdir('Data - Brick models/')
brick_models = []

with open("brick_models.pkl", 'rb') as fp:
    brick_models = pickle.load(fp)
    

In [4]:
total_instances = {}
for model in brick_models:
    total_instances[model] = get_total_num_instances(model, 'brick:Equipment') + get_total_num_instances(model, 'brick:Location') + get_total_num_instances(model, 'brick:Point')

In [14]:
# for file in files:
#     if file.endswith(".ttl"):
#         g = brickschema.Graph(load_brick=True)
#         g.load_file('Data - Brick models/'+file)
#         g.expand(profile="owlrl")
#         brick_models.append(g)

# with open("brick_models.pkl", 'wb') as fp:
#     pickle.dump(brick_models, fp)


In [9]:
g=query_metrics[q1[4]]['graph']
for i, node in g.nodes().data():
    print(node)

{'query': [['?sensor', 'rdf:type', 'brick:Zone_Air_Temperature_Sensor'], ['?sensor', 'brick:isPointOf', '?equip']], 'node_id': 0, 'level': 0}
{'query': (['?sensor', 'rdf:type', 'brick:Zone_Air_Temperature_Sensor'], ['?sensor', 'brick:isPointOf', '?equip']), 'node_id': 1, 'level': 0}
{'query': (['?sensor', 'rdf:type', 'brick:Zone_Air_Temperature_Sensor'], ['?sensor', 'brick:isPointOf+', '?equip']), 'node_id': 2, 'level': 1}
{'query': (['?sensor', 'rdf:type', 'brick:Air_Temperature_Sensor'], ['?sensor', 'brick:isPointOf', '?equip']), 'node_id': 3, 'level': 1}
{'query': (['?sensor', 'rdf:type', 'brick:Air_Temperature_Sensor'], ['?sensor', 'brick:isPointOf+', '?equip']), 'node_id': 4, 'level': 2}
{'query': (['?sensor', 'rdf:type', 'brick:Temperature_Sensor'], ['?sensor', 'brick:isPointOf', '?equip']), 'node_id': 5, 'level': 2}
{'query': (['?sensor', 'rdf:type', 'brick:Temperature_Sensor'], ['?sensor', 'brick:isPointOf+', '?equip']), 'node_id': 6, 'level': 3}
{'query': (['?sensor', 'rdf:typ

In [6]:
max_level = 5
query_metrics = {}
for query in q1:
    print(query)
    start_time = datetime.datetime.now()
    G = get_fast_relaxed_graph(query=query, max_level=max_level)
    end_time = datetime.datetime.now()
    query_metrics[query]={'graph': G}
    query_metrics[query]['relaxation_time']  = (end_time-start_time).total_seconds()
    query_metrics[query]['query_size'] = len(extract_triples(query))
    print(query_metrics[query])

SELECT * WHERE { ?s ?p ?o }
{'graph': <networkx.classes.graph.Graph object at 0x7ffb4d82f7f0>, 'relaxation_time': 0.000433, 'query_size': 1}
SELECT ?point ?point_type WHERE {
    ?point rdf:type brick:Point .
    ?point rdf:type ?point_type 
}
{'graph': <networkx.classes.graph.Graph object at 0x7ffb4d82fe50>, 'relaxation_time': 0.000155, 'query_size': 2}
SELECT ?meter WHERE { ?meter rdf:type brick:Green_Button_Meter }
{'graph': <networkx.classes.graph.Graph object at 0x7ffbdb936eb0>, 'relaxation_time': 0.000108, 'query_size': 1}
 SELECT ?t WHERE { ?t rdf:type brick:Weather_Temperature_Sensor }
{'graph': <networkx.classes.graph.Graph object at 0x7ffb4d82f1c0>, 'relaxation_time': 0.000441, 'query_size': 1}
SELECT ?sensor WHERE {
    ?sensor rdf:type brick:Zone_Air_Temperature_Sensor .
    ?sensor brick:isPointOf ?equip 
}
{'graph': <networkx.classes.graph.Graph object at 0x7ffbdb93eca0>, 'relaxation_time': 0.000656, 'query_size': 2}
SELECT ?sp WHERE {
    ?sp rdf:type brick:Zone_Air_Temp

In [5]:
buildings = brick_models

In [6]:
metrics = {}
for query in q1[:-1]:
    relaxed_graph = query_metrics[query]['graph']
    select_statement = query.split("{")[0] + "{\n"
    
    level_building_nodes = {}
    max_edits = -1
    for level in range(max_level+1):

        building_nodes_with_data = {}
        for building_idx in range(len(buildings)):
            building_model = buildings[building_idx]
            nodes_with_data= []

            for node_idx, node in relaxed_graph.nodes().data():
                if node['level'] == level:
                    
                    if max_edits == -1:
                        max_edits = level
                    elif level>max_edits:
                        max_edits = level
                        
                    brick_query = generate_brick_query_from_node(node['query'], select_statement=select_statement)
                    res = run_brick_query(building_model=building_model, query=brick_query)

                    if len(res) > 0:
                        nodes_with_data.append(node)
            building_nodes_with_data[building_idx] = len(nodes_with_data)
        level_building_nodes[level] = building_nodes_with_data
        query_metrics[query]['level_building_num_nodes_with_data'] = level_building_nodes
        query_metrics[query]['size_relaxed_graph'] = len(relaxed_graph.nodes())
        query_metrics[query]['max_edits'] = max_edits
        metrics[query] = {
            'level_building_num_nodes_with_data': level_building_nodes,
            'size_relaxed_graph': len(relaxed_graph.nodes()),
            'max_edits': max_edits,
            'query_size': query_metrics[query]['query_size'],
            'relaxation_time': query_metrics[query]['relaxation_time']
        }
    print(query_metrics[query])
    with open("metrics_all_buildings.json", 'w') as fp:
        json.dump(metrics, fp)

{'graph': <networkx.classes.graph.Graph object at 0x7fd609ed8b20>, 'relaxation_time': 0.00042, 'query_size': 1, 'level_building_num_nodes_with_data': {0: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1}, 1: {0: 4, 1: 6, 2: 6, 3: 6, 4: 6, 5: 6, 6: 6, 7: 6, 8: 2, 9: 4, 10: 2, 11: 6, 12: 6, 13: 4, 14: 4, 15: 6, 16: 6, 17: 4, 18: 6, 19: 4, 20: 6, 21: 4, 22: 6, 23: 4, 24: 2, 25: 6, 26: 4, 27: 6, 28: 6, 29: 6, 30: 4, 31: 6, 32: 6, 33: 2, 34: 6, 35: 6, 36: 6, 37: 4, 38: 4, 39: 6, 40: 6, 41: 6, 42: 6, 43: 2, 44: 6, 45: 2, 46: 4, 47: 4, 48: 6, 49: 6}, 2: {0: 4, 1: 6, 2: 6, 3: 6, 4: 6, 5: 6, 6: 6, 7: 6, 8: 2, 9: 4, 10: 2, 11: 6, 12: 6, 13: 4, 14: 4, 15: 6, 16: 6, 17: 4, 18: 6, 19: 4, 20: 6, 21: 4, 22: 6, 23: 

In [1]:
'a'

'a'

In [4]:
# file = "Data - Brick models/artx.ttl"

q1 = """SELECT * WHERE {
 ?sen   rdf:type   brick:Supply_Air_Temperature_Sensor .
 ?sp    rdf:type   brick:Supply_Air_Temperature_Setpoint .
 ?ahu   rdf:type   brick:AHU .
 ?ahu   brick:feeds ?thing .
 ?thing  brick:hasPoint ?sen .
 ?thing  brick:hasPoint ?sp .
}"""


In [ ]:
q1_res = get_fast_evaluation_metrics(file="Data - Brick models/acad.ttl", query=q1, max_level=5)
q1_res

In [12]:
c = pymortar.Client("https://beta-api.mortardata.org")
print(q1)
sites = c.qualify([q1]).sites

SELECT ?damper ?vav WHERE {
    ?damper   rdf:type   brick:Damper . 
    ?vav    rdf:type   brick:VAV . 
    ?damper   brick:isPartOf  ?vav   
}


In [16]:
df = c.qualify([q1])

In [ ]:
brickschema.